# Synthea Case Study
*Lab 26 April 2022*

* https://synthea.mitre.org/
* https://github.com/synthetichealth/synthea

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Patients;

CREATE TABLE Patients(
    id VARCHAR(36),
    birthdate DATE,
    deathdate DATE,
    ssn VARCHAR(11),
    drivers VARCHAR(9),
    passport VARCHAR(15),
    prefix VARCHAR(4),
    first VARCHAR(20),
    last VARCHAR(20),
    suffix VARCHAR(5),
    maiden VARCHAR(20),
    marital CHAR,
    race VARCHAR(10),
    ethnicity VARCHAR(15),
    gender CHAR,
    birthplace VARCHAR(100),
    address VARCHAR(50),
    city VARCHAR(30),
    state VARCHAR(30),
    county VARCHAR(50),
    zip VARCHAR(10),
    lat DECIMAL(18,15),
    lon DECIMAL(18,15),
    healthcare_expenses VARCHAR(20),
    healthcare_coverage VARCHAR(20),
    PRIMARY KEY(id)
) AS SELECT id,birthdate,deathdate,ssn,drivers,passport,prefix,first,last,suffix,maiden,marital,race,ethnicity,gender,
            birthplace,address,city,state,county,zip,lat,lon,healthcare_expenses,healthcare_coverage
FROM CSVREAD('../../data/synthea/scenario02/csv/patients.csv');

DROP TABLE IF EXISTS Encounters;

CREATE TABLE Encounters(
    id VARCHAR(36),
    start DATETIME,
    stop DATETIME,
    patient VARCHAR(36),
    organization VARCHAR(36),
    provider VARCHAR(36),
    payer  VARCHAR(36),
    encounterclass VARCHAR(20),
    code VARCHAR(15),
    description VARCHAR(100),
    base_encounter_cost DECIMAL(8,2),
    total_claim_cost DECIMAL(10,2),
    payer_coverage DECIMAL(8,2),
    reasoncode VARCHAR(15),
    reasondescription VARCHAR(100),
    PRIMARY KEY(id)
) AS SELECT id,start,stop,patient,organization,provider,payer,encounterclass,code,description,base_encounter_cost,
            total_claim_cost,payer_coverage,reasoncode,reasondescription
FROM CSVREAD('../../data/synthea/scenario02/csv/encounters.csv');

DROP TABLE IF EXISTS Conditions;

CREATE TABLE Conditions(
    start DATETIME,
    stop DATETIME,
    patient VARCHAR(36),
    encounter VARCHAR(36),
    code VARCHAR(20),
    description_condition VARCHAR(100),
) AS SELECT start,stop,patient,encounter,code,description
FROM CSVREAD('../../data/synthea/scenario02/csv/conditions.csv');

DROP TABLE IF EXISTS Careplans;

CREATE TABLE Careplans(
    id VARCHAR(36),
    start DATETIME,
    stop DATETIME,
    patient VARCHAR(36),
    encounter VARCHAR(36),
    code VARCHAR(20),
    description VARCHAR(100),
    reasoncode VARCHAR(15),
    reasondescription VARCHAR(100),
) AS SELECT id,start,stop,patient,encounter,code,description,reasoncode,reasondescription
FROM CSVREAD('../../data/synthea/scenario02/csv/careplans.csv');


## Listing Patients

In [3]:
SELECT id, to_char(birthdate,'dd-mm-yyyy') birth, to_char(deathdate,'dd-mm-yyyy') death,
       deathdate-birthdate days_life
FROM Patients
LIMIT 5;

## Number of Dead Patients

In [4]:
SELECT COUNT(*) FROM Patients WHERE deathdate IS NOT NULL;

121

## Listing Encounters

In [5]:
SELECT id, to_char(start,'dd-mm-yyyy') start, encounterclass, description
FROM Encounters
LIMIT 5;

# Encounters in the Emergency

## Number of Patients that Visited the Emergency and Urgentcare



In [6]:
SELECT COUNT(DISTINCT patient) FROM Encounters
       WHERE (encounterclass = 'emergency' OR encounterclass = 'urgentcare');

824

## Number of Patients with Cerebrovascular Accident (Code 230690007)

In [7]:
SELECT count(patient), patient, p.first, p.last FROM Encounters e, patients p
       WHERE e.patient = p.id AND
       --(encounterclass = 'emergency' OR encounterclass = 'urgentcare') AND
       (code='230690007')
      group by e.patient
     -- LIMIT 5;

## Last Encounter in the Emergency

In [8]:
DROP VIEW IF EXISTS Emergency;

CREATE VIEW Emergency AS
SELECT patient, MAX(start) last_encounter
       FROM Encounters --depois fazer um join com a tabela de pacientes
       WHERE encounterclass = 'urgentcare' OR encounterclass = 'emergency'
       GROUP BY patient;

## Conditions in the Last Encounter

## Patients that had a Stroke in the Last Encounter

In [9]:
SELECT COUNT(distinct e.patient)--, e.patient, c.code, c.description_condition
       FROM Emergency e, Conditions c, Patients p
       WHERE e.patient = c.patient AND
             c.patient = p.id AND
           -- (c.description_condition='Stroke');
             c.code='230690007'
      -- group by e.patient

42

# Patients with a Stroke that Died 30 days after the Last Encounter

In [10]:
SELECT COUNT(*)
       FROM Patients p, Emergency s, Conditions c
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 30 AND
             (c.description_condition='Stroke');

SELECT DISTINCT p.first, p.last,
                to_char(p.birthdate,'dd-mm-yyyy') birth, to_char(p.deathdate,'dd-mm-yyyy') death,
                (p.deathdate-p.birthdate)/365 age,
                to_char(s.last_encounter,'dd-mm-yyyy') last_encounter,
                p.deathdate-s.last_encounter days
       FROM Patients p, Emergency s, Conditions c
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 30 AND
             (c.description_condition='Stroke')
            --LIMIT 5;

# Patients with a Stroke that Died 1 year after the Last Encounter

In [11]:
SELECT COUNT(*)
       FROM Patients p, Emergency s, Conditions c
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 365 AND
             (c.description_condition='Stroke');

SELECT DISTINCT p.first, p.last,
                to_char(p.birthdate,'dd-mm-yyyy') birth, to_char(p.deathdate,'dd-mm-yyyy') death,
                (p.deathdate-p.birthdate)/365 age,
                to_char(s.last_encounter,'dd-mm-yyyy') last_encounter,
                p.deathdate-s.last_encounter days
       FROM Patients p, Emergency s, Conditions c
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 365 AND
             (c.description_condition='Stroke') order by first, last
           --  LIMIT 5;

## Cross information with the patient that has a Careplan

In [12]:
SELECT count(patient) FROM Careplans
       WHERE (reasoncode = '59621000' --Hypertension
              OR reasoncode = '736285004') --Hyperlipidemia
       

294

In [ ]:
SELECT COUNT(distinct(e.patient))
       FROM Encounters e, Careplans cp, Conditions c
       WHERE (e.patient = cp.patient) AND (cp.patient = c.patient)
       AND (c.description_condition='Stroke') -- Quantos pacientes estao utilizando plano de saude para tratar Stroke

40

## Cross information with the total amount of expenses in Healthcare and Race by patient

In [ ]:
SELECT e.patient, p.HEALTHCARE_EXPENSES, p.deathdate, p.race
       FROM Encounters e, Careplans cp, Conditions c, Patients p, Emergency s
       WHERE (e.patient = cp.patient) AND (cp.patient = c.patient) AND (p.id = c.patient)
       AND (c.description_condition='Stroke') AND p.id=s.patient
       LIMIT 5;

## HEADACHE HYPOTESIS


In [ ]:
DROP VIEW IF EXISTS OnlyStrokeEmergency;

CREATE VIEW OnlyStrokeEmergency AS 
SELECT distinct(e.patient), MAX(e.start) last_encounter
       FROM Encounters e, Conditions c
       WHERE e.patient = c.patient and
            c.code='230690007'
       GROUP BY e.patient;

In [ ]:
SELECT COUNT(*) from OnlyStrokeEmergency

42

In [ ]:
DROP TABLE IF EXISTS EmergencyML;

CREATE TABLE EmergencyML (
  patient VARCHAR(36),
  first VARCHAR(20),
  last VARCHAR(20),
  last_encounter DATETIME,
  deathdate DATETIME,
    
  race VARCHAR(10),
  ETHNICITY VARCHAR(15),
  GENDER CHAR,
    
  hadstroke SMALLINT DEFAULT 0,
  death07days SMALLINT DEFAULT 0,
  death30days SMALLINT DEFAULT 0,
  death90days SMALLINT DEFAULT 0,
  death365days SMALLINT DEFAULT 0,
  PRIMARY KEY(patient)
) AS
SELECT s.patient,p.first, p.last, MAX(last_encounter), p.deathdate,p.RACE,p.ETHNICITY,p.GENDER,0,0,0,0,0
       FROM Emergency s, Patients p, Conditions c
       WHERE s.patient = c.patient and 
             c.patient = p.id /*and
             c.description_condition='Stroke'*/
       GROUP BY s.patient, first, last;
       
SELECT count(1) FROM EmergencyML order by first, last;



In [ ]:
UPDATE EmergencyML e
SET e.hadstroke = 1
WHERE EXISTS
(SELECT c.patient
 FROM Conditions c
 WHERE c.patient=e.patient AND c.description_condition='Stroke');

In [ ]:
Select count(*) 
FROM EmergencyML e
Where e.hadstroke = 1


In [ ]:
SELECT *
       FROM EmergencyML 
       WHERE hadstroke = 1 order by first, last

In [ ]:
SELECT COUNT(*)
       FROM EmergencyML
       WHERE hadstroke = 1

In [ ]:
UPDATE EmergencyML e
SET e.death07days = 1
WHERE EXISTS
(SELECT c.patient
 FROM Patients p, Conditions c
 WHERE c.patient=e.patient AND 
       e.patient = p.id    AND
       c.description_condition='Stroke' AND
       p.deathdate IS NOT NULL AND
       p.deathdate-e.last_encounter <= 7);

In [ ]:
Select count(*) 
FROM EmergencyML e
Where e.death07days = 1;

In [ ]:
UPDATE EmergencyML e
SET e.death30days = 1
WHERE EXISTS
(SELECT c.patient
 FROM Patients p, Conditions c
 WHERE c.patient=e.patient AND 
       e.patient = p.id    AND
       c.description_condition='Stroke' AND
       p.deathdate IS NOT NULL AND
       p.deathdate-e.last_encounter <= 30);


In [ ]:
Select * 
FROM EmergencyML e
Where e.death30days = 1 order by first, last;

In [ ]:
Select count(*) 
FROM EmergencyML e
Where e.death30days = 1;

In [ ]:
UPDATE EmergencyML e
SET e.death90days = 1
WHERE EXISTS
(SELECT c.patient
 FROM Patients p, Conditions c
 WHERE c.patient=e.patient AND 
       e.patient = p.id    AND
       c.description_condition='Stroke' AND
       p.deathdate IS NOT NULL AND
       p.deathdate-e.last_encounter between 31 AND 90 --AND
);

In [ ]:
Select * 
FROM EmergencyML e
Where e.death90days = 1

In [ ]:
Select count(*) 
FROM EmergencyML e
Where e.death90days = 1

In [ ]:
UPDATE EmergencyML e
SET e.death365days = 1
WHERE EXISTS
(SELECT c.patient
 FROM Patients p, Conditions c
 WHERE c.patient=e.patient AND 
       e.patient = p.id    AND
       c.description_condition='Stroke' AND
       p.deathdate IS NOT NULL AND
       p.deathdate-e.last_encounter between 91 AND 365 --AND
);

In [ ]:
Select * 
FROM EmergencyML e
Where e.death365days = 1;

In [ ]:
Select count(*) 
FROM EmergencyML e
Where e.death365days = 1;

In [ ]:
select * from EmergencyML
limit 5;

In [ ]:
CALL CSVWRITE('../../data/synthea/scenario02/csv-ml/emergency-ml.csv', 'SELECT * FROM EmergencyML');

In [ ]:
/*UPDATE EmergencyML e
SET e.hadstroke = 1
WHERE EXISTS
(SELECT c.patient
 FROM Patients p, Conditions c
 WHERE c.patient=e.patient AND 
       e.patient = p.id    AND
       c.description_condition='Stroke' AND
       p.deathdate IS NOT NULL AND
       p.deathdate-e.last_encounter <= 30 AND
);



SELECT COUNT(*)
       FROM Patients p, Conditions c--, Emergency s
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 365 AND
             (c.description_condition='Stroke');

SELECT DISTINCT p.first, p.last,
                to_char(p.birthdate,'dd-mm-yyyy') birth, to_char(p.deathdate,'dd-mm-yyyy') death,
                (p.deathdate-p.birthdate)/365 age,
                to_char(s.last_encounter,'dd-mm-yyyy') last_encounter,
                p.deathdate-s.last_encounter days
       FROM Patients p, Emergency s, Conditions c
       WHERE p.id=s.patient AND s.patient=c.patient AND
             p.deathdate IS NOT NULL AND p.deathdate-s.last_encounter <= 365 AND
             (c.description_condition='Stroke')*/

In [ ]:
/*SELECT COUNT(DISTINCT c.patient)
       FROM  Conditions c, OnlyStrokeEmergency O
       WHERE c.patient = o.patient AND
             c.code = '73595000' */

In [ ]:
/*Select * 
FROM EmergencyML e
Where e.death90days = 1
LIMIT 5;*/

In [ ]:
/*Select * 
FROM EmergencyML e
Where e.death365days = 1
LIMIT 5;*/

In [ ]:
/*SELECT COUNT(DISTINCT e.patient)
       FROM Emergency e, Conditions c
       WHERE e.patient = c.patient AND
            (c.code ='230690007');*/